In [1]:
"""


Original paper - https://arxiv.org/abs/1611.08024



"""

'\n\n\nOriginal paper - https://arxiv.org/abs/1611.08024\n\n\n\n'

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


# Load in Hospital Data

In [35]:
import pickle
from collections import Counter

dir = "E:/ChristianMusaeus/Data/Eyes_closed_marked"
# with open(f"{dir}/df_EOandEC_norm.pkl", "rb") as f:
#     data = pickle.load(f)

# load df_EOandEC_norm.csv
data = pd.read_csv(f"{dir}/df_EOandEC_norm.csv")


print(data.shape)


epochs = data.iloc[:, :-2].values
labels = data.iloc[:, -2].values
patient_ids = data.iloc[:, -1].values
print(f"min max of epochs: {np.min(epochs)} {np.max(epochs)}")

# Assuming 'labels' is a NumPy array or a Pandas Series
label_counts = Counter(labels)
print("Label Distribution:", label_counts)


(3215, 3802)
min max of epochs: -1.0 1.0000000000000004
Label Distribution: Counter({1.0: 1740, 0.0: 1475})


# EEGNet 8,2 from paper


In [29]:
class EEGNetModel(nn.Module):  # EEGNET-8,2
    def __init__(self, chans=19, classes=1, time_points=200, temp_kernel=100,
                 f1=8, f2=16, d=2, pk1=4, pk2=4, dropout_rate=0.25, max_norm1=1, max_norm2=0.25):
        super(EEGNetModel, self).__init__()


        # Temporal Filters
        self.block1 = nn.Sequential(
            nn.Conv2d(1, f1, (1, temp_kernel), padding='same', bias=False),  # Temporal Conv
            #nn.BatchNorm2d(f1),
        )
        # Spatial Filters
        self.block2 = nn.Sequential(
            nn.Conv2d(f1, d * f1, (chans, 1), groups=f1, bias=False),  # Depthwise Conv
            #nn.BatchNorm2d(d * f1),
            nn.ELU(),
            nn.AvgPool2d((1, pk1)),  # Pool over time dimension
            nn.Dropout(dropout_rate)
        )
        # Separable Convolution
        self.block3 = nn.Sequential(
            nn.Conv2d(d * f1, f2, (1, 16), padding='same', bias=False, groups=f2),  # Separable Conv
            nn.Conv2d(f2, f2, kernel_size=1, bias=False),  # Pointwise Conv
            #nn.BatchNorm2d(f2),
            nn.ELU(),
            nn.AvgPool2d((1, pk2)),  # Pool over time dimension
            nn.Dropout(dropout_rate)
        )
        

        # Fully Connected
        self.fc = nn.Linear(16 * ( time_points // 16 ), classes)

    #     # Apply max_norm constraint to the depthwise layer in block2
        self._apply_max_norm(self.block2[0], max_norm1)

    def _apply_max_norm(self, layer, max_norm):
        for name, param in layer.named_parameters():
            if 'weight' in name:
                param.data = torch.renorm(param.data, p=2, dim=0, maxnorm=max_norm)

    def forward(self, x):
        # Input x is (batch_size,1, num_channels, num_timepoints)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x


# Adjust input size to (19x200)
sample_len = 200  # Your time points
channels = 19     # Your number of channels

# summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# check if CUDA is available
print(device)
torch.cuda.is_available()
torch.cuda.get_device_name(0)

model = EEGNetModel(chans=channels, time_points=sample_len).to(device)
def initialize_weights(m):
    # if isinstance(m, nn.Conv2d):
    #     nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

model.apply(initialize_weights)
# summary(model, (1, 19, 200))
for param in model.parameters():
    param.requires_grad = True
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter '{name}' does require gradients.")

print(model.forward(torch.rand(13, 1, channels, sample_len).to(device)))
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-1)

cuda
Parameter 'block1.0.weight' does require gradients.
Parameter 'block2.0.weight' does require gradients.
Parameter 'block3.0.weight' does require gradients.
Parameter 'block3.1.weight' does require gradients.
Parameter 'fc.weight' does require gradients.
Parameter 'fc.bias' does require gradients.
tensor([[0.4892],
        [0.4984],
        [0.5034],
        [0.4991],
        [0.4898],
        [0.5043],
        [0.4924],
        [0.5144],
        [0.4843],
        [0.5198],
        [0.4894],
        [0.4877],
        [0.4941]], device='cuda:0', grad_fn=<SigmoidBackward0>)


# Dataset Class

In [30]:
from torch.utils.data import DataLoader, Dataset

# Dataset class definition
class EEGDataset(Dataset):
    def __init__(self, epochs, labels):
        self.df = epochs
        self.labels = labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Access the epoch and corresponding label
        data = self.df[idx]  # data shape: (3800)
        data = data.reshape(19,200)  # shape: (19, 200)
        label = self.labels[idx]  # label shape: (1)
        
        # Ensure the data is converted to float tensor
        data_tensor = torch.tensor(data, dtype=torch.float32).to(device)
        
        # Convert label to tensor (assumed to be scalar)
        label_tensor = torch.tensor(label, dtype=torch.float32).to(device)
        
        return data_tensor, label_tensor

# Create EEGDataset object and print a batch of 13 samples
dataset = EEGDataset(epochs, labels)
dataloader = DataLoader(dataset, batch_size=13, shuffle=True)

print(next(iter(dataloader))[0].shape)
print(next(iter(dataloader))[0])


torch.Size([13, 19, 200])
tensor([[[ 0.0668,  0.0847, -0.0190,  ...,  0.1303,  0.0131, -0.0404],
         [ 0.0822,  0.0262,  0.0271,  ...,  0.1911,  0.1588,  0.1474],
         [-0.0876, -0.2015, -0.2951,  ..., -0.0743,  0.1558,  0.2586],
         ...,
         [-0.3060, -0.4222, -0.4577,  ..., -0.1926, -0.0146,  0.0190],
         [-0.4316, -0.4187, -0.2916,  ...,  0.2159,  0.3337,  0.2565],
         [ 0.0723,  0.1230,  0.2307,  ...,  0.1630,  0.2122,  0.2382]],

        [[-0.4750, -0.4920, -0.4821,  ..., -0.6150, -0.6232, -0.6291],
         [-0.6037, -0.6205, -0.6297,  ..., -0.7322, -0.7447, -0.7550],
         [-0.5677, -0.5738, -0.5064,  ..., -0.6604, -0.6525, -0.6652],
         ...,
         [-0.1045, -0.0857, -0.0542,  ..., -0.4604, -0.4797, -0.3754],
         [ 0.0874,  0.1148,  0.1577,  ...,  0.6608,  0.6371,  0.6486],
         [-0.1550, -0.1375, -0.0636,  ...,  0.3729,  0.3225,  0.2637]],

        [[-0.2913, -0.2600, -0.3066,  ..., -0.3229, -0.2664, -0.3023],
         [-0.2011, 

# Training and evaluation loop

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
from tqdm import tqdm
import torch
import numpy as np

# define random seed
torch.manual_seed(13)

# multiply data with a coefficient
coefs = [10000,1000,100,10, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]
epochs_copy = epochs.copy()
epochs_copy *= coefs[2]


batch_size = 32
num_epochs = 50
patience = 20  # Stop if no improvement after 20 epochs
best_val_acc = 0.0  # Track the best validation accuracy
no_improvement_epochs = 0  # Track how many epochs without improvement

# Lists to store loss values for plotting later
train_loss_history = []
val_loss_history = []

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

model = EEGNetModel().to(device)

# patient_ids = np.repeat([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], len(epochs) // 10)  # Adjust the length accordingly
patient_ids = patient_ids

for epoch in tqdm(range(num_epochs)):  # Loop over the number of epochs
    with torch.enable_grad():
        # for name, param in model.named_parameters():
        #     if param.requires_grad:
        #         print(f"Parameter '{name}' does require gradients.")

        print(f"Epoch {epoch+1}")
        running_loss = []
        running_val_loss = []

        for train_index, test_index in logo.split(epochs_copy, labels, groups=patient_ids):
            

            # Train and test split
            X_train, X_val = epochs_copy[train_index, :], epochs_copy[test_index, :]
            y_train, y_val = labels[train_index], labels[test_index]
            
            # Create DataLoader for training and testing
            train_dataset = EEGDataset(X_train, y_train)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

            val_dataset = EEGDataset(X_val, y_val)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            
            # Training
            model.train()
            for i, data in enumerate(train_loader, 0):
                # Get the inputs; data is a list of [inputs, labels/targets]
                inputs, targets = data
                inputs, targets = inputs.to(device), targets.to(device)

                
                # check if the input shape is correct
                inputs = inputs.unsqueeze(1)  # Add a channel dimension if needed
                # print(f"input shape: {inputs.shape}")

            
                # Forward pass
                outputs = model(inputs)
                targets = targets.unsqueeze(1)  # Ensure targets have the same shape as outputs
                loss = criterion(outputs, targets)
                
                # Backward pass and optimization
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clip gradients to prevent exploding gradients
                # for name, param in model.named_parameters():
                #     if param.requires_grad and param.grad is not None:
                #         print(f"{name}: Grad Mean = {param.grad.mean()}, Grad Std = {param.grad.std()}")
                        # if torch.isnan(param.grad).any():
                        #     print(f"NaN detected in gradients of {name}")
                        
                optimizer.step()
                # Zero the parameter gradients
                optimizer.zero_grad()


                running_loss.append(loss.item())

        # Validation Loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        accuracies = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                # inputs, targets = Variable(inputs), Variable(targets)
                inputs = inputs.unsqueeze(1)  # Add a channel dimension if needed

                outputs = model(inputs)
                print(outputs)
                probabilities = torch.sigmoid(outputs)
                predicted = (probabilities > 0.5).float()
                
                loss = criterion(outputs, targets.unsqueeze(1))
                running_val_loss.append(loss.item())

                # Calculate validation accuracy
                total += targets.size(0)
                correct += (predicted.squeeze() == targets).sum().item()

        accuracy = 100 * correct / total
        accuracies.append(accuracy)
        avg_val_loss = np.mean(running_val_loss)
        
        # Print losses and accuracy for current epoch
        print(f'Epoch {epoch+1}, Validation Accuracy: {accuracy} %')
        print(f"Training Loss: {np.mean(running_loss):.4f}, Validation Loss: {avg_val_loss:.4f}")
        
        # Store losses for plotting later
        train_loss_history.append(np.mean(running_loss))
        val_loss_history.append(avg_val_loss)

        # Early Stopping logic
        if accuracy > best_val_acc:
            best_val_acc = accuracy  # Update the best validation accuracy
            no_improvement_epochs = 0  # Reset the counter if there's improvement
            torch.save(model.state_dict(), f"{dir}/EEGNet_model.pth")  # Save the best model
        else:
            no_improvement_epochs += 1

        if no_improvement_epochs >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break  # Exit the training loop


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1


  2%|▏         | 1/50 [00:15<12:15, 15.02s/it]

tensor([[0.3007],
        [0.4937],
        [0.3260],
        [0.2332],
        [0.2146],
        [0.1343],
        [0.4484],
        [0.3216],
        [0.7602],
        [0.4605],
        [0.4365],
        [0.3613],
        [0.2756],
        [0.4044],
        [0.3571],
        [0.4049],
        [0.3948],
        [0.1923],
        [0.2472],
        [0.3182],
        [0.2723],
        [0.4650],
        [0.4390],
        [0.3068],
        [0.4114],
        [0.2760],
        [0.5743],
        [0.5657],
        [0.3513],
        [0.5886],
        [0.2417],
        [0.4963]], device='cuda:0')
tensor([[0.4583],
        [0.4662],
        [0.4407],
        [0.3093],
        [0.4055],
        [0.3643],
        [0.3675],
        [0.4699],
        [0.3842],
        [0.3738],
        [0.1894],
        [0.5020],
        [0.2495],
        [0.4012],
        [0.3089],
        [0.6017],
        [0.3977],
        [0.4260],
        [0.4558],
        [0.4011],
        [0.4932],
        [0.4747],
        [0

  4%|▍         | 2/50 [00:30<12:11, 15.23s/it]

tensor([[0.3007],
        [0.4937],
        [0.3260],
        [0.2332],
        [0.2146],
        [0.1343],
        [0.4484],
        [0.3216],
        [0.7602],
        [0.4605],
        [0.4365],
        [0.3613],
        [0.2756],
        [0.4044],
        [0.3571],
        [0.4049],
        [0.3948],
        [0.1923],
        [0.2472],
        [0.3182],
        [0.2723],
        [0.4650],
        [0.4390],
        [0.3068],
        [0.4114],
        [0.2760],
        [0.5743],
        [0.5657],
        [0.3513],
        [0.5886],
        [0.2417],
        [0.4963]], device='cuda:0')
tensor([[0.4583],
        [0.4662],
        [0.4407],
        [0.3093],
        [0.4055],
        [0.3643],
        [0.3675],
        [0.4699],
        [0.3842],
        [0.3738],
        [0.1894],
        [0.5020],
        [0.2495],
        [0.4012],
        [0.3089],
        [0.6017],
        [0.3977],
        [0.4260],
        [0.4558],
        [0.4011],
        [0.4932],
        [0.4747],
        [0

  6%|▌         | 3/50 [00:45<11:51, 15.14s/it]

tensor([[0.3007],
        [0.4937],
        [0.3260],
        [0.2332],
        [0.2146],
        [0.1343],
        [0.4484],
        [0.3216],
        [0.7602],
        [0.4605],
        [0.4365],
        [0.3613],
        [0.2756],
        [0.4044],
        [0.3571],
        [0.4049],
        [0.3948],
        [0.1923],
        [0.2472],
        [0.3182],
        [0.2723],
        [0.4650],
        [0.4390],
        [0.3068],
        [0.4114],
        [0.2760],
        [0.5743],
        [0.5657],
        [0.3513],
        [0.5886],
        [0.2417],
        [0.4963]], device='cuda:0')
tensor([[0.4583],
        [0.4662],
        [0.4407],
        [0.3093],
        [0.4055],
        [0.3643],
        [0.3675],
        [0.4699],
        [0.3842],
        [0.3738],
        [0.1894],
        [0.5020],
        [0.2495],
        [0.4012],
        [0.3089],
        [0.6017],
        [0.3977],
        [0.4260],
        [0.4558],
        [0.4011],
        [0.4932],
        [0.4747],
        [0

  6%|▌         | 3/50 [00:48<12:43, 16.25s/it]


KeyboardInterrupt: 

# Dont look under here

In [ ]:
from sklearn.model_selection import train_test_split

# Split data once before training
X_train, X_val, y_train, y_val = train_test_split(
    epochs, labels, test_size=0.2, random_state=42, stratify=labels
)

train_dataset = EEGDataset(X_train, y_train)
val_dataset = EEGDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

for epoch in tqdm(range(num_epochs)):  # Loop over the number of epochs
    print(f"Epoch {epoch+1}")
    running_loss = []
    running_val_loss = []

    # Training Phase
    model.train()
    for i, data in enumerate(train_loader, 0):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.unsqueeze(1)  # Add channel dimension if needed

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Monitor gradients
        for name, param in model.named_parameters():
            # if param.requires_grad and param.grad is not None:
            #     grad_mean = param.grad.mean().item()
            #     grad_std = param.grad.std().item()
            #     print(f"{name}: Grad Mean = {grad_mean}, Grad Std = {grad_std}")
                if torch.isnan(param.grad).any():
                    print(f"NaN detected in gradients of {name}")
                # chewck if the gradients are positive
                if (param.grad > 0).any():
                    print(f"Positive gradients detected in {name}")

        optimizer.step()

        running_loss.append(loss.item())

    # Validation Phase
    model.eval()
    correct = 0
    total = 0
    running_val_loss = []
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            inputs = inputs.unsqueeze(1)  # Add channel dimension if needed

            outputs = model(inputs)
            probabilities = torch.sigmoid(outputs)
            predicted = (probabilities > 0.5).float()
            
            loss = criterion(outputs, targets.unsqueeze(1))
            running_val_loss.append(loss.item())

            correct += (predicted.squeeze() == targets).sum().item()
            total += targets.size(0)

    accuracy = 100 * correct / total
    avg_train_loss = np.mean(running_loss)
    avg_val_loss = np.mean(running_val_loss)

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {accuracy:.2f}%')

    # Early Stopping
    if accuracy > best_val_acc:
        best_val_acc = accuracy
        no_improvement_epochs = 0
        torch.save(model.state_dict(), f"{dir}/EEGNet_model.pth")
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f" no improvement :{no_improvement_epochs}")
        print(f"Early stopping at epoch {epoch+1}")
        break

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected i

  2%|▏         | 1/50 [00:00<00:32,  1.52it/s]

Epoch 1, Train Loss: 0.6915, Val Loss: 0.6909, Val Acc: 54.59%
Epoch 2
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive grad

  4%|▍         | 2/50 [00:01<00:30,  1.59it/s]

...

 24%|██▍       | 12/50 [00:07<00:21,  1.73it/s]

Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2

 26%|██▌       | 13/50 [00:07<00:21,  1.69it/s]

Epoch 13, Train Loss: 0.6914, Val Loss: 0.6909, Val Acc: 54.59%
Epoch 14
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gradients detected in block1.0.weight
Positive gradients detected in block2.0.weight
Positive gradients detected in block3.0.weight
Positive gradients detected in block3.1.weight
Positive gradients detected in fc.weight
Positive gr

KeyboardInterrupt: 

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Please check your installation.")

    import numpy as np

print(f"Any NaNs in data: {np.isnan(epochs).any()}")
print(f"Any Infs in data: {np.isinf(epochs).any()}")

Using device: cuda
CUDA Version: 11.8
CUDA Device Name: NVIDIA GeForce RTX 3060 Ti
Any NaNs in data: False
Any Infs in data: False


In [ ]:
# PLOTTING
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(train_loss_history, label='Training Loss')
plt.plot(val_loss_history, label='Validation Loss')
plt.title('EEGNet - Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, len(accuracies)+1), accuracies, label='Validation Accuracy')  # Adjust accordingly
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Validation Accuracy Over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# Load Turkish data

In [ ]:
# load the dataframe from the pickle file
import pickle
dir = "E:/ChristianMusaeus/Data/Turkish"
with open(f"{dir}/dataframe.pkl", "rb") as f:
    df = pickle.load(f)

# keep these channels only, these are the indexes: 
""" 
Fp1 -> 0
Fp2 -> 33
F3  -> 4
F4  -> 38
C3  -> 12
C4  -> 48
P3  -> 20
P4  -> 55
O1  -> 26
O2  -> 61
F7  -> 6
F8  -> 40
T7  -> 14
T8  -> 50
P7  -> 22
P8  -> 57
Fz  -> 36
Cz  -> 46
Pz  -> 30 

but add 1 to each index, since the first channel is channel_1
"""

df = df[["channel_1", "channel_34", "channel_5", "channel_39", "channel_13", "channel_49", "channel_21", "channel_56", "channel_27", "channel_62", "channel_7", "channel_41", "channel_15", "channel_51", "channel_23", "channel_58", "channel_37", "channel_47", "channel_31", "label"]]

patient_ids = np.repeat([1,2,3,4,5,6,7,8,9,10],76288)  # Make sure to have this aligned with your epochs/labels

# Normalize per patient (within training and test sets)
data = df.drop("label", axis=1).values
data_norm = []
for patient_id in np.unique(patient_ids):
    patient_data = data[patient_ids == patient_id]
    scaler = StandardScaler()
    patient_data_scaled = scaler.fit_transform(patient_data)
    print(f"min and max of patient {patient_id}: {np.min(patient_data_scaled), np.max(patient_data_scaled)}")
    data_norm.append(patient_data_scaled)

data_norm = np.concatenate(data_norm, axis=0)
# add labels back
y = df["label"].values
data = data_norm

# make data a dataframe again
df = pd.DataFrame(data)
print(f"size of df: {df.shape}")

# split the data into epochs of 256 datapoints each
epochs = []
for i in range(0, len(df), 256):
    epochs.append(df.iloc[i:i+256].values)

# convert the list of epochs to a numpy array
epochs = np.array(epochs)
print(f"size of epochs: {epochs.shape}")

# save the SCALED epochs
with open(f"{dir}/scaled_rawEEG_epochs.pkl", "wb") as f:
    pickle.dump(epochs, f)




min and max of patient 1: (np.float64(-2.5061664365916747), np.float64(3.2048110636610154))
min and max of patient 2: (np.float64(-2.2856199344616766), np.float64(3.1704214250154332))
min and max of patient 3: (np.float64(-2.6244058798465795), np.float64(2.326236025125405))
min and max of patient 4: (np.float64(-2.9839669159521462), np.float64(3.0033738566092985))
min and max of patient 5: (np.float64(-2.7180645928577865), np.float64(1.9617189652654394))
min and max of patient 6: (np.float64(-3.8235823428136357), np.float64(1.5538075585523163))
min and max of patient 7: (np.float64(-2.17836904765987), np.float64(2.700097334829556))
min and max of patient 8: (np.float64(-1.9391087092804788), np.float64(4.868952462591622))
min and max of patient 9: (np.float64(-3.278040789072896), np.float64(2.18437991679923))
min and max of patient 10: (np.float64(-1.6112388744411374), np.float64(4.696875071895533))
size of df: (762880, 19)
size of epochs: (2980, 256, 19)


In [ ]:
# open the scaled epochs
import pickle
import pandas as pd
import torch
dir = "E:/ChristianMusaeus/Data/Turkish"

with open(f"{dir}/scaled_rawEEG_epochs.pkl", "rb") as f:
    epochs = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")
# convert the numpy array to a tensor and move it to the GPU
epochs = torch.tensor(epochs, dtype=torch.float32).to(device)
print()
# print(torch.rand(2,3).cuda())



device: cuda



In [ ]:
labels = []
for _ in range(10):
    labels.append(np.repeat([1, 0], 149))
labels = np.concatenate(labels)

print(f"size of labels: {labels.shape}")


size of labels: (2980,)
